In [ ]:
# ! pip install scanpy
# ! pip install torch


In [ ]:
import torch
from vega_model import VEGA
from utils import *
from learning_utils import *
import scanpy as sc
from scipy import sparse
from sklearn import preprocessing
import numpy as np
import itertools
import argparse
import os
import pandas as pd

In [ ]:
train_file_list=os.listdir('./data')
train_file_list=[i for i in train_file_list if '.h5ad' in i]

In [ ]:
train_file_list

In [ ]:
train_data = sc.read('./data/'+train_file_list[0])

In [ ]:
pathway_file_list=os.listdir('./pathway')
pathway_file_list=[i for i in pathway_file_list if '.gmt' in i]

In [ ]:
pathway_file_list

In [ ]:
pathway_dict = read_gmt('./pathway/'+pathway_file_list[0], min_g=0, max_g=1000)
print(train_data.shape, flush=True)

In [ ]:
pathway_mask = create_pathway_mask(list(train_data.var.index), pathway_dict, add_missing=1, fully_connected=True)

In [ ]:
path_way_ind=[]
for i in range(pathway_mask.shape[0]):
    path_way_ind.append(np.argmax(pathway_mask[i,:]))

In [ ]:
print(path_way_ind) # check all pathway not one the same dimension

In [ ]:
labels = train_data.obs['label'].values
le = preprocessing.LabelEncoder()
le.fit(labels)
y = torch.Tensor(le.transform(labels))

In [ ]:
train_ds = torch.Tensor(train_data.X)
train_ds = UnsupervisedDataset(train_ds,y)

In [ ]:
LR = 1e-4
N_EPOCHS=500
p_drop = 0.5
dir_name='./save_model/'
model_prefix=pathway_file_list[0].split('.gmt')[0].split('.')[0].split('_')[0]+'_GBM' # change the number of pathway_file_list , the number should be the same with you choose before

In [ ]:
kfold = KFoldTorch(cv=10, n_epochs=N_EPOCHS, lr=LR, train_p=10, test_p=10, num_workers=0, save_all=True, save_best=False, path_dir=dir_name, model_prefix='go__all')
dict_params = {'pathway_mask':pathway_mask, 'beta':0.0001, 'dropout':p_drop, 'path_model':None, 'device':'cpu', 'positive_decoder':True}
kfold.train_kfold(VEGA, dict_params, train_ds, batch_size=2000)
np.save(dir_name+'5CV_vega_organoid.npy', kfold.cv_res_dict)



In [ ]:
help(UnsupervisedDataset)